In [3]:
# Import necessary libraries
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, to_timestamp, hour, dayofweek
import matplotlib.pyplot as plt

In [4]:
# Create a SparkSession
spark = SparkSession.builder \
    .appName("NYC Taxi Data Analysis") \
    .getOrCreate()

In [ ]:
# Load the Parquet file (replace with your actual path)
df = spark.read.parquet("data/yellow_tripdata_2025-01.parquet")

In [ ]:
# Show schema and first few rows
df.printSchema()
df.show(5)